In [1]:
%cd /home/ubuntu/Qwen-Image-Edit-Angles

/home/ubuntu/Qwen-Image-Edit-Angles


In [2]:
import huggingface_hub 
from qwenimage.datamodels import QwenConfig
from qwenimage.foundation import QwenImageFoundationSaveInterm
from datasets import concatenate_datasets, load_dataset, interleave_datasets

/usr/lib/python3/dist-packages/sklearn/utils/fixes.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version  # type: ignore
2025-11-24 16:55:55.657889: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-24 16:55:55.671869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764003355.688913 3244532 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/home/ubuntu/.local/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.12) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Skipping import of cpp extensions due to incompatible torch version 2.9.1+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
TMA benchmarks will be running without grid constant TMA descriptor.
ERROR:bitsandbytes.cextension:Could not load bitsandbytes native library: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
  File "/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
  File "/usr/lib/python3.10/ctypes/__init__.py", line 452, in LoadLibrary
    return self._dlltype(name)
  File "/usr/lib/python3.10/ctypes/__init__.py", line 374, in __init__
    self._ha

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
repo_tree = huggingface_hub.list_repo_tree(
    "WeiChow/CrispEdit-2M",
    "data",
    repo_type="dataset",
)

all_paths = []
for i in repo_tree:
    all_paths.append(i.path)

parquet_prefixes = set()
for path in all_paths:
    if path.endswith('.parquet'):
        filename = path.split('/')[-1]
        if '_' in filename:
            prefix = filename.split('_')[0]
            parquet_prefixes.add(prefix)

print("Found parquet prefixes:", sorted(parquet_prefixes))


In [3]:
total_per = 1

EDIT_TYPES = [
    "color",
    "style",
    "replace",
    "remove",
    "add",
    "motion change",
    "background change",
]

In [4]:


all_edit_datasets = []
for edit_type in EDIT_TYPES:
    to_concat = []
    for ds_n in range(total_per):
        ds = load_dataset("parquet", data_files=f"/data/CrispEdit/{edit_type}_{ds_n:05d}.parquet", split="train")
        to_concat.append(ds)
    edit_type_concat = concatenate_datasets(to_concat)
    all_edit_datasets.append(edit_type_concat)

# consistent ordering for indexing, also allow extension by increasing total_per
join_ds = interleave_datasets(all_edit_datasets)

In [5]:
for d in join_ds:
    print(d["instruction"])

Turn dress positioned in the lower central area into red with floral patterns
Please recreate this in digital painting artistic style
replace the largemouth bass with a trout
remove the four grey extraterrestrials observing the scene
Add the lid back onto the jar of white cream
The character stops exhaling smoke and closes their mouth.
change the background to a mystical forest
Turn cap positioned in the upper-central area into red
Draw this image using fantasy art technique
replace the alien spaceship with a futuristic submarine
remove the morbidly obese Frankenstein monster from the school bus
add a halved lemon
No change in the subject's action, pose, or facial expression.
change the background to a serene starry night sky
Turn baby positioned in the central area into darker skin tone
Convert this image to graffiti artwork
replace the pregnant woman with an elderly woman holding a walking stick
remove the Wakandan technology elements from the townscape
add a large, serene pool area


KeyboardInterrupt: 

In [ ]:
from pathlib import Path


save_base_dir = Path("/data/regression_output")
save_base_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
foundation = QwenImageFoundationSaveInterm(QwenConfig())

In [ ]:
import torch


for idx, input_data in enumerate(join_ds):

    output_dict = foundation.base_pipe(foundation.INPUT_MODEL(
        image=[input_data["input_img"]],
        prompt=input_data["instruction"],
    ))

    torch.save(output_dict, save_base_dir/f"{idx:06d}.pt")


In [ ]:
output_dict = torch.load(save_base_dir/f"{idx:06d}.pt", weights_only=False)

In [ ]:
output_dict.keys()

In [ ]:
test_ind = 10

latents_i_start = output_dict[f"latents_{test_ind}_start"]
t_i = output_dict[f"t_{test_ind}"]
v_i = output_dict[f"noise_pred_{test_ind}"]

proj_out_i = latents_i_start - t_i * v_i

In [ ]:
proj_out_i_1d = proj_out_i
proj_out_i_2d = foundation.unpack_latents(proj_out_i_1d, output_dict["height"] // 16, output_dict["width"] // 16, )
proj_out_i_pil = foundation.latents_to_pil(proj_out_i_2d)
proj_out_i_pil[0]

In [ ]:
out_1d = output_dict["image_latents"]
out_2d = foundation.unpack_latents(out_1d, output_dict["height"] // 16, output_dict["width"] // 16, )
out_pil = foundation.latents_to_pil(out_2d)
# out_pil[0]
# join_ds[idx]["input_img"]
# join_ds[idx]["instruction"]

In [ ]:
proj_out_i_pil[0].size